In [ ]:
%run -i model.ipynb

## Deep GP for density estimation

http://adamian.github.io/talks/Damianou_NIPS17.pdf

http://bayesiandeeplearning.org/2017/papers/69.pdf

Recovering 

$$p(y | \mu)$$

$$p(r | \mu)$$


1. Add DeepKernelLearning

2. Add RealNVP to output(?)



In [ ]:
import pyro
import pyro.distributions as dist
import pyro.contrib.gp as gp
import torch
from torch import nn

import pyro.infer as infer

In [ ]:
deep_kernel_nn = nn.Sequential(nn.Linear(1, 10), nn.ReLU(), 
                            nn.Linear(10, 10), nn.ReLU(), 
                            nn.Linear(10, 1))

In [ ]:
rbf = gp.kernels.RBF(input_dim=1)

In [ ]:
deep_kernel_pyro = lambda x: pyro.module("deep_kernel", deep_kernel_nn)(x)

In [ ]:
deep_kernel_pyro = gp.kernels.Warping(rbf, deep_kernel_pyro)

In [ ]:
likelihood = gp.likelihoods.Gaussian(variance=torch.tensor(1.))

In [ ]:
mu_range = (-50, 50)

In [ ]:
x = ((mu_range[0] - mu_range[1]) * (torch.rand(10000)) + mu_range[1]) #.to(device)

with pyro.condition(data={"mu": x}):
    y = y_sample().detach()

In [ ]:
gpmodule = gp.models.GPRegression(X=x.view(-1, 1), y=y, 
                                   kernel=deep_kernel_pyro)
                                   # likelihood=likelihood,
                                  #whiten=True)

In [ ]:
# optimizer = torch.optim.Adam(gpmodule.parameters(), lr=1e-4)
# gp.util.train(gpmodule, optimizer)

In [ ]:
optimizer = torch.optim.Adam(gpmodule.parameters(), lr=1e-1)
elbo = infer.TraceMeanField_ELBO()
loss_fn = elbo.differentiable_loss
# csis = infer.CSIS(gpmodule.model, gpmodule.guide, optimizer)

In [ ]:
for i in range(1000):
    optimizer.zero_grad()
    loss = loss_fn(gpmodule.model, gpmodule.guide)
    loss.backward()
    optimizer.step()
    if i % 50 == 0:
        print(loss.item())

In [ ]:
loc, sd = gpmodule(gpmodule.X)

In [ ]:
plt.scatter(gpmodule.X.detach().cpu().numpy().ravel(), loc.detach().cpu().numpy())

In [ ]:
plt.scatter(gpmodule.X.detach().cpu().numpy().ravel(), gpmodule.y.detach().cpu().numpy())

In [ ]:
loss

In [ ]:
gp.kernels.Sum()